In [1]:
import f3dasm
import numpy as np
from numdifftools import Gradient
from dataclasses import dataclass
import copy
from typing import Tuple

# Third-party
import autograd
import autograd.core
import autograd.numpy as np
import tensorflow as tf
from autograd import elementwise_grad as egrad

from f3dasm.optimization.adapters.tensorflow_implementations import SimpelModel, convert_autograd_to_tensorflow

2022-12-19 12:39:23.368187: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-19 12:39:23.512075: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-12-19 12:39:24.175903: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.1/lib64
2022-12-19 12:39:24.175968: W tensorflow/compiler/xla/stream

In [2]:
dimensionality = 2
design = f3dasm.make_nd_continuous_design(bounds=np.tile([-1.,1.], (dimensionality,1)), dimensionality=dimensionality)
function = f3dasm.functions.Sphere(dimensionality=dimensionality, scale_bounds=design.get_bounds())

x = np.array([0.3,0.4])

In [5]:
args = {}

args["model"] = SimpelModel(
    None,
    args={
        "dim": function.dimensionality,
        "x0": np.zeros(function.dimensionality),
    },
)  # Build the model
args["tvars"] = args["model"].trainable_variables
args["func"] = convert_autograd_to_tensorflow(function.__call__)


2022-12-19 12:39:34.157642: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-12-19 12:39:34.157734: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (martin): /proc/driver/nvidia/version does not exist
2022-12-19 12:39:34.159270: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [14]:
function.dfdx(x)

array([46.46406779, 58.86281222])

In [9]:
def dfdx2(x):
    args["model"].z.assign(x)

    with tf.GradientTape() as tape:
        tape.watch(args["tvars"])
        logits = 0.0 + tf.cast(args["model"](None), tf.float64)
        loss = args["func"](tf.reshape(
            logits, (function.dimensionality)))

    grads = tape.gradient(loss, args["tvars"])
    return grads[0].numpy().copy()

In [10]:
args["func"]

In [11]:
args["tvars"]

[<tf.Variable 'Variable:0' shape=(2,) dtype=float32, numpy=array([0.3, 0.4], dtype=float32)>]

In [12]:
args["model"].z.assign(np.array([0.22, 0.4]))

<tf.Variable 'UnreadVariable' shape=(2,) dtype=float32, numpy=array([0.22, 0.4 ], dtype=float32)>

In [13]:
args["model"].z

<tf.Variable 'Variable:0' shape=(2,) dtype=float32, numpy=array([0.22, 0.4 ], dtype=float32)>